 1. Registro en Kaggle y Elección de Competencia:
 Elegir una competencia activa que esté alineada con los conocimientos y técnicas aprendidas.
 La competencia activa elegida es:
https://www.kaggle.com/competitions/playground-series-s5e7

Objetivo: Predecir si una persona es introvertida o extrovertida, considerando su comportamiento social y rasgos de personalidad.
Tipo: Clasificación binaria.

2. Carga y Exploración Inicial de Datos:

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report, roc_curve

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Importamos el DataFrame.
df = pd.read_csv("/content/drive/MyDrive/Skillnest/ML/CORES/data_challenge.csv", sep=",")

In [24]:
# Visualizamos nuestro DataFrame.
df. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18524 entries, 0 to 18523
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         18524 non-null  int64  
 1   Time_spent_Alone           17334 non-null  float64
 2   Stage_fear                 16631 non-null  object 
 3   Social_event_attendance    17344 non-null  float64
 4   Going_outside              17058 non-null  float64
 5   Drained_after_socializing  17375 non-null  object 
 6   Friends_circle_size        17470 non-null  float64
 7   Post_frequency             17260 non-null  float64
 8   Personality                18524 non-null  object 
dtypes: float64(5), int64(1), object(3)
memory usage: 1.3+ MB


In [25]:
df.head()

,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


 3. Análisis Exploratorio de Datos (EDA):

In [26]:
# Tipos de variables
# Categoricas
df.select_dtypes(include="object").columns.tolist()

['Stage_fear', 'Drained_after_socializing', 'Personality']

In [27]:
# Tipos de variables
# Numericas
df.select_dtypes(include=["int64", "float64"]).columns.tolist()

['id',
 'Time_spent_Alone',
 'Social_event_attendance',
 'Going_outside',
 'Friends_circle_size',
 'Post_frequency']

In [28]:
# Descripción de variables numéricas
df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
id,18524.0,9261.50,5347.56,0.0,4630.75,9261.5,13892.25,18523.0
Time_spent_Alone,17334.0,3.14,3.00,0.0,1.00,2.0,4.00,11.0
Social_event_attendance,17344.0,5.27,2.75,0.0,3.00,5.0,8.00,10.0
Going_outside,17058.0,4.04,2.06,0.0,3.00,4.0,6.00,7.0
Friends_circle_size,17470.0,8.00,4.22,0.0,5.00,8.0,12.00,15.0
Post_frequency,17260.0,4.98,2.88,0.0,3.00,5.0,7.00,10.0


Time_spent_Alone: En promedio, las personas pasan unas 3.14 unidades de tiempo. La desviación es alta (3), indicando mucha variabilidad: algunos pasan poco tiempo solos (mínimo 0), otros mucho (máximo 11). La mayoría pasa entre 1 y 4.

Social_event_attendance: El promedio es 5.27, lo que indica que la mayoría asiste a eventos sociales con frecuencia media-alta (máximo es 10). El rango es amplio, con algunos que nunca asisten (0) y otros que asisten mucho (10).

Going_outside: En promedio 4.04, con valores mínimos en 0 y máximo en 7. Indica cuán frecuentemente salen fuera de casa.

Friends_circle_size: En promedio, tienen un círculo social de tamaño 8, con un rango de 0 (sin amigos) a 15 (muy grande). La desviación es relativamente alta, mostrando diversidad social.

Post_frequency: Frecuencia media de publicación de aproximadamente 5 (sobre una escala máxima 10), con gran variabilidad. Algunos no publican nada y otros publican mucho.

In [29]:
# Descripción de variables categóricas
df.describe(include="object").T

,count,unique,top,freq
Stage_fear,16631,2,No,12609
Drained_after_socializing,17375,2,No,13313
Personality,18524,2,Extrovert,13699


Stage_fear (Miedo escénico):
Dos categorías: "No" y otra (probablemente "Yes").
La mayoría (12,609) no tiene miedo escénico.
Hay alrededor de 1,172 valores nulos.
Esto puede indicar que la mayoría no tiene este miedo, pero hay un grupo relevante que sí.

Drained_after_socializing (Cansancio tras socializar):
También dos categorías: "No" y otra (probablemente "Yes").
La mayoría (13,313) responde que "No", no se siente agotada después de socializar.
Hay 1,149 valores nulos.

Personality (Personalidad):
"Extrovert" y "Introvert".
La mayoría de la muestra es extrovertida (13,699) frente a 5,825 introvertidos.

In [30]:
# Verificar valores nulos
df.isna().sum()

,0
id,0
Time_spent_Alone,1190
Stage_fear,1893
Social_event_attendance,1180
Going_outside,1466
Drained_after_socializing,1149
Friends_circle_size,1054
Post_frequency,1264
Personality,0


In [31]:
# Porcentaje de nulos por columna
porcentaje_nulos = df.isna().mean().sort_values(ascending=False) * 100

# Mostrar en forma de tabla
porcentaje_nulos = porcentaje_nulos.round(2).reset_index()
porcentaje_nulos.columns = ["Columna", "Porcentaje de Nulos"]
porcentaje_nulos

,Columna,Porcentaje de Nulos
0,Stage_fear,10.22
1,Going_outside,7.91
2,Post_frequency,6.82
3,Time_spent_Alone,6.42
4,Social_event_attendance,6.37
5,Drained_after_socializing,6.20
6,Friends_circle_size,5.69
7,id,0.00
8,Personality,0.00


Todas las variables tienen entre 5.7% y 10.2% de valores nulos, lo cual es moderado pero importante de manejar.

Justificación para imputar valores faltantes:

Variables numéricas: Se imputaron los valores faltantes con la mediana de cada columna. La mediana es una medida de tendencia central.

Variables categóricas: Se imputaron los valores faltantes con la moda (valor más frecuente) de cada columna. La moda representa la categoría más común y es una forma simple y efectiva de reemplazar valores faltantes en variables categóricas.

In [32]:
# Variables numéricas
num_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']

# Variables categóricas
cat_cols = ['Stage_fear', 'Drained_after_socializing']


In [33]:
# Imputar variables numéricas con mediana
median_values = df[num_cols].median()
df[num_cols] = df[num_cols].fillna(median_values)

In [34]:
# Imputar variables categóricas con moda
mode_values = df[cat_cols].mode().iloc[0]
df[cat_cols] = df[cat_cols].fillna(mode_values)


In [35]:
# Verificar valores nulos
df.isna().sum()

,0
id,0
Time_spent_Alone,0
Stage_fear,0
Social_event_attendance,0
Going_outside,0
Drained_after_socializing,0
Friends_circle_size,0
Post_frequency,0
Personality,0


In [36]:
# Verificar filas duplicadas
df.duplicated().sum()

np.int64(0)

No existen duplicados.

Análisis posibles outliers:
Según la interpretación de las estadisticas básicas, las categorias con posibles outliers son:

Time_spent_Alone
Social_event_attendance
Friends_circle_size
Post_frequency

Estas se evaluaran mediante boxplot.

In [37]:
# Variables numéricas con posibles outliers
cols_box = ["Time_spent_Alone", "Social_event_attendance", "Friends_circle_size", "Post_frequency"]

# Generar un boxplot por cada variable
for col in cols_box:
    fig = px.box(
        df,
        y=col,
        points="all",
        title=f"Boxplot de {col} con puntos de outliers"
    )
    fig.update_layout(title_font_size=18)
    fig.show()

1. Time_spent_Alone
Mediana (línea central): Aprox. 2.
IQR (25% a 75%): Entre 1 y 4.
Outliers: Hay bastantes valores por sobre 7–11, que están muy lejos del rango intercuartílico.

Algunos individuos pasan considerablemente más tiempo solos que la mayoría. Podrían ser casos extremos o errores.

2. Social_event_attendance
Mediana: Aprox. 5.
IQR: Entre 3 y 8.
Outliers: Hay pocos puntos cercanos al valor máximo (10), pero no extremos.
La mayoría asiste a eventos sociales en una frecuencia media, aunque hay unos pocos que lo hacen de forma muy frecuente, lo cual puede reflejar rasgos fuertemente extrovertidos.

3. Friends_circle_size
Mediana: Aprox. 8.
IQR: Entre 5 y 12.
Outliers: Hay valores mínimos en 0 (sin amigos), lo que puede ser un dato real o un caso extremo.
Aunque la mayoría tiene un tamaño de círculo social moderado, algunos tienen cero amigos registrados (posibles introvertidos extremos o valores atípicos), y otros llegan al máximo (15).

Post_frequency
Mediana: Aprox. 5.
IQR: Entre 3 y 7.
Outliers: Algunos valores mínimos (0 publicaciones) y máximos (10).
Aunque la mayoría tiene una frecuencia media de publicaciones, hay usuarios que no publican nada o publican mucho, lo que puede reflejar comportamientos sociales extremos.


1. Time_spent_Alone
Valores extremos altos, pueden ser:
Personas muy introvertidas. lo cual es válido.
Representan un comportamiento real, personas que están solas todo el tiempo.

2. Social_event_attendance
Escala esperada:
Outliers están cerca del límite superior (frecuencia alta).
Son personas muy activas socialmente.

3. Friends_circle_size
El valor 0 puede parecer un outlier, pero es realista, personas sin amigos.
El valor 0 puede ser clave para distinguir a personas solitarias.

4. Post_frequency
El valor 0 puede parecer un outlier, pero también es natural, alguien que no publica nada.

 A modo de conclusión final, no eliminare ni reemplazare ningún outlier.
Todos los valores están dentro de rangos esperados y son coherentes con el comportamiento humano.

 4. Preprocesamiento de Datos:

La variable objetivo original, Personality, contiene dos categorías: "Extrovert" e "Introvert". Estas categorías ya están definidas, lo que convierte este caso en una clasificación binaria supervisada.

Personality es categórica nominal, por ende, se crea la variable Personality_1 para transformar la variable categórica original Personality en un formato binario.
Extrovert:	0
Introvert: 1

Varibles:
Númericas: "Time_spent_Alone", "Social_event_attendance", "Going_outside","Friends_circle_size" y "Post_frequency"
Nominales: "Stage_fear" y "Drained_after_socializing"

Variable a eliminar: "id", porque no aporta información predictiva.

In [38]:
# Si es introvertido, dara true y se convierte en 1, de lo contrario sera false y se convierte en 0 (Extrovert)
df["Personality_1"] = (df["Personality"] == "Introvert").astype(int)

In [39]:
# Definir features y target
X = df.drop(columns=["id", "Personality", "Personality_1"])  # Eliminamos identificador, clase original y codificada
y = df["Personality_1"]  # Variable objetivo binaria

In [40]:
# Dividir en train y test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
# Definir transformadores.
col_num = ["Time_spent_Alone", "Social_event_attendance", "Going_outside", "Friends_circle_size", "Post_frequency"]
col_nom = ["Stage_fear", "Drained_after_socializing"]

 5. Implementación de Modelos y Benchmark:

Regresión Logística

In [42]:
# Preprocesador para Regresión Logística
preprocessor_rl = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [43]:
# Pipeline RL
pipeline_rl = Pipeline(steps=[
    ("preprocessor", preprocessor_rl),
    ("classifier", LogisticRegression(max_iter=1000))
])

In [44]:
# Entrenar el modelo
pipeline_rl.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Time_spent_Alone',
                                                   'Social_event_attendance',
                                                   'Going_outside',
                                                   'Friends_circle_size',
                                                   'Post_frequency']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Stage_fear',
                                                   'Drained_after_socializing'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [45]:
# Predecir
y_pred_rl = pipeline_rl.predict(X_test)

In [46]:
# Evaluar
accuracy_rl = accuracy_score(y_test, y_pred_rl)

K-Nearest Neighbors (KNN)

In [48]:
# Preprocesador para KNN
preprocessor_knn = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [49]:
# Pipeline KNN
pipeline_knn = Pipeline(steps=[
    ("preprocessor", preprocessor_knn),
    ("classifier", KNeighborsClassifier())
])

In [50]:
# Entrenar el modelo
pipeline_knn.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Time_spent_Alone',
                                                   'Social_event_attendance',
                                                   'Going_outside',
                                                   'Friends_circle_size',
                                                   'Post_frequency']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Stage_fear',
                                                   'Drained_after_socializing'])])),
                ('classifier', KNeighborsClassifier())])

In [51]:
# Predecir
y_pred_knn = pipeline_knn.predict(X_test)

In [52]:
# Evaluar
accuracy_knn = accuracy_score(y_test, y_pred_knn)

Árbol de Decisión

In [54]:
# Preprocesador para Árbol de Decisión
preprocessor_tree = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [55]:
# Pipeline para Árbol de Decisión
pipeline_tree = Pipeline(steps=[
    ("preprocessor", preprocessor_tree),
    ("classifier", DecisionTreeClassifier(random_state=42))
])

In [56]:
# Entrenar el modelo
pipeline_tree.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Time_spent_Alone',
                                                   'Social_event_attendance',
                                                   'Going_outside',
                                                   'Friends_circle_size',
                                                   'Post_frequency']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Stage_fear',
                                                   'Drained_after_socializing'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])

In [57]:
# Predecir
y_pred_tree = pipeline_tree.predict(X_test)

In [58]:
# Evaluar
accuracy_tree = accuracy_score(y_test, y_pred_tree)

XGBoost

In [59]:
# Preprocesador para XGBoost
preprocessor_xgb = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [60]:
# Pipeline para XGBoost
pipeline_xgb = Pipeline(steps=[
    ("preprocessor", preprocessor_xgb),
    ("classifier", XGBClassifier(random_state=42))
])

In [61]:
# Entrenar el modelo
pipeline_xgb.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Time_spent_Alone',
                                                   'Social_event_attendance',
                                                   'Going_outside',
                                                   'Friends_circle_size',
                                                   'Post_frequency']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Stage_fear',
                                                   'Drained_after_socializing'])])),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [62]:
# Predecir
y_pred_xgb = pipeline_xgb.predict(X_test)

In [63]:
# Evaluar
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

LightGBM

In [66]:
# Preprocesador para LightGBM
preprocessor_lgbm = ColumnTransformer(transformers=[
    ("num", StandardScaler(), col_num),
    ("nom", OneHotEncoder(handle_unknown="ignore"), col_nom)
])

In [67]:
# Pipeline para LightGBM
pipeline_lgbm = Pipeline(steps=[
    ("preprocessor", preprocessor_lgbm),
    ("classifier", LGBMClassifier(random_state=42))
])

In [68]:
# Entrenar el modelo
pipeline_lgbm.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



[LightGBM] [Info] Number of positive: 3873, number of negative: 10946
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001455 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 71
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.261354 -> initscore=-1.038945
[LightGBM] [Info] Start training from score -1.038945


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Time_spent_Alone',
                                                   'Social_event_attendance',
                                                   'Going_outside',
                                                   'Friends_circle_size',
                                                   'Post_frequency']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Stage_fear',
                                                   'Drained_after_socializing'])])),
                ('classifier', LGBMClassifier(random_state=42))])

In [69]:
# Predecir
y_pred_lgbm = pipeline_lgbm.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [70]:
# Evaluar
accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)

In [71]:
# Resultados
print(f"Accuracy Regresión Logística: {accuracy_rl:.4f}")
print(f"Accuracy KNN: {accuracy_knn:.4f}")
print(f"Accuracy Árbol de Decisión: {accuracy_tree:.4f}")
print(f"Accuracy XGBoost: {accuracy_xgb:.4f}")
print(f"Accuracy LightGBM : {accuracy_lgbm:.4f}")

Accuracy Regresión Logística: 0.9684
Accuracy KNN: 0.9690
Accuracy Árbol de Decisión: 0.9344
Accuracy XGBoost: 0.9665
Accuracy LightGBM : 0.9682


Todos los modelos excepto el Árbol de Decisión muestran excelente rendimiento (sobre 96%).
KNN mejor rendimiento inicial con 96.90% de accuracy.
Regresión Logística segundo mejor con 96.84%, muy similar a KNN.
LightGBM también con alta precisión de 96.82%.
XGBoost con un accuracy de 96.65%, apenas por debajo de los anteriores.
Árbol de Decisión claramente el más débil con 93.44%.

Optimizar los hiperparámetros de los modelos seleccionados utilizando GridSearchCV

Regresión Logística Optimización

In [72]:
# Definir parámetros
param_grid_rl = {
    "classifier__C": [0.01, 0.1, 1, 10],
    "classifier__penalty": ["l2"]
}
grid_rl = GridSearchCV(pipeline_rl, param_grid=param_grid_rl, cv=5, scoring="accuracy")
grid_rl.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Time_spent_Alone',
                                                                          'Social_event_attendance',
                                                                          'Going_outside',
                                                                          'Friends_circle_size',
                                                                          'Post_frequency']),
                                                                        ('nom',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Stage_fear',
                                                                          'Drained_after_socializing'])])),
                                       ('classifier',
                                        LogisticRegression(max_iter=1000))]),
             param_grid={'classifier__C': [0.01, 0.1, 1, 10],
                         'classifier__penalty': ['l2']},
             scoring='accuracy')

In [73]:
# Evaluación con el mejor estimador
best_rl = grid_rl.best_estimator_
y_pred_rl1 = best_rl.predict(X_test)

K-Nearest Neighbors (KNN) Optimización

In [75]:
# Definir parámetros
param_grid_knn = {
    "classifier__n_neighbors": [3, 5, 7, 9],
}
grid_knn = GridSearchCV(pipeline_knn, param_grid=param_grid_knn, cv=5, scoring="accuracy")
grid_knn.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Time_spent_Alone',
                                                                          'Social_event_attendance',
                                                                          'Going_outside',
                                                                          'Friends_circle_size',
                                                                          'Post_frequency']),
                                                                        ('nom',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Stage_fear',
                                                                          'Drained_after_socializing'])])),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid={'classifier__n_neighbors': [3, 5, 7, 9]},
             scoring='accuracy')

In [76]:
# Evaluación con el mejor estimador
best_knn = grid_knn.best_estimator_
y_pred_knn1 = best_knn.predict(X_test)

Árbol de Decisión - Optimización

In [77]:
# Definir parámetros
param_grid_tree = {
    "classifier__max_depth": [3, 5, 10, None],
}

grid_tree = GridSearchCV(pipeline_tree, param_grid=param_grid_tree, cv=5, scoring="accuracy")
grid_tree.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Time_spent_Alone',
                                                                          'Social_event_attendance',
                                                                          'Going_outside',
                                                                          'Friends_circle_size',
                                                                          'Post_frequency']),
                                                                        ('nom',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Stage_fear',
                                                                          'Drained_after_socializing'])])),
                                       ('classifier',
                                        DecisionTreeClassifier(random_state=42))]),
             param_grid={'classifier__max_depth': [3, 5, 10, None]},
             scoring='accuracy')

In [78]:
# Evaluación con el mejor estimador
best_tree = grid_tree.best_estimator_
y_pred_tree1 = best_tree.predict(X_test)

XGBoost - Optimización

In [79]:
# Definir parámetros
param_grid_xgb = {
    "classifier__n_estimators": [50, 100, 150],
    "classifier__max_depth": [3, 5, 7],
}

grid_xgb = GridSearchCV(pipeline_xgb, param_grid=param_grid_xgb, cv=5, scoring="accuracy")
grid_xgb.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Time_spent_Alone',
                                                                          'Social_event_attendance',
                                                                          'Going_outside',
                                                                          'Friends_circle_size',
                                                                          'Post_frequency']),
                                                                        ('nom',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Stage_fear',
                                                                          'Drained_after_socializing'])])),
                                       ('classifier',
                                        XGBClassifier(base_score...
                                                      max_cat_threshold=None,
                                                      max_cat_to_onehot=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      multi_strategy=None,
                                                      n_estimators=None,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      random_state=42, ...))]),
             param_grid={'classifier__max_depth': [3, 5, 7],
                         'classifier__n_estimators': [50, 100, 150]},
             scoring='accuracy')

In [80]:
# Evaluación con el mejor estimador
best_xgb = grid_xgb.best_estimator_
y_pred_xgb1 = best_xgb.predict(X_test)

LightGBM - Optimización

In [81]:
# Definir parámetros
param_grid_lgbm = {
    "classifier__n_estimators": [50, 100, 150],
    "classifier__max_depth": [3, 5, 7],
}
grid_lgbm = GridSearchCV(pipeline_lgbm, param_grid=param_grid_lgbm, cv=5, scoring="accuracy")
grid_lgbm.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



[LightGBM] [Info] Number of positive: 3099, number of negative: 8756
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000923 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 71
[LightGBM] [Info] Number of data points in the train set: 11855, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.261409 -> initscore=-1.038660
[LightGBM] [Info] Start training from score -1.038660
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



[LightGBM] [Info] Number of positive: 3099, number of negative: 8757
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000894 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 71
[LightGBM] [Info] Number of data points in the train set: 11856, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.261387 -> initscore=-1.038774
[LightGBM] [Info] Start training from score -1.038774


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



[LightGBM] [Info] Number of positive: 3873, number of negative: 10946
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001084 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 71
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.261354 -> initscore=-1.038945
[LightGBM] [Info] Start training from score -1.038945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Time_spent_Alone',
                                                                          'Social_event_attendance',
                                                                          'Going_outside',
                                                                          'Friends_circle_size',
                                                                          'Post_frequency']),
                                                                        ('nom',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Stage_fear',
                                                                          'Drained_after_socializing'])])),
                                       ('classifier',
                                        LGBMClassifier(random_state=42))]),
             param_grid={'classifier__max_depth': [3, 5, 7],
                         'classifier__n_estimators': [50, 100, 150]},
             scoring='accuracy')

In [82]:
# Evaluación con el mejor estimador
best_lgbm = grid_lgbm.best_estimator_
y_pred_lgbm1 = best_lgbm.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [83]:
# Resultados optimizados
print("Regresión logistica optimizado")
print("Mejores parámetros:", grid_rl.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_rl1))

print("KNN Classifier")
print("Mejores parámetros:", grid_knn.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_knn1))

print("Arbol de decision")
print("Mejores parámetros:", grid_tree.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_tree1))

print("XGBoost")
print("Mejores parámetros:", grid_xgb.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_xgb1))

print("LightGBM")
print("Mejores parámetros:", grid_lgbm.best_params_)
print("Accuracy:", accuracy_score(y_test, y_pred_lgbm1))

Regresión logistica optimizado
Mejores parámetros: {'classifier__C': 1, 'classifier__penalty': 'l2'}
Accuracy: 0.968421052631579
KNN Classifier
Mejores parámetros: {'classifier__n_neighbors': 9}
Accuracy: 0.9686909581646423
Arbol de decision
Mejores parámetros: {'classifier__max_depth': 5}
Accuracy: 0.9678812415654521
XGBoost
Mejores parámetros: {'classifier__max_depth': 3, 'classifier__n_estimators': 100}
Accuracy: 0.9686909581646423
LightGBM
Mejores parámetros: {'classifier__max_depth': 5, 'classifier__n_estimators': 100}
Accuracy: 0.9689608636977058


Regresión Logística:
No hubo un cambio significativo en el accuracy tras optimizar el parámetro C. Esto indica que  ya estaba bastante bien ajustada en el modelo sin optimizar.

KNN:
Mejoró levemente al ajustar n_neighbors a 9.

Árbol de Decisión:
La profundidad óptima resultó ser 5.

XGBoost:
Usó 100 árboles y profundidad 3 como mejor configuración, resultando en un rendimiento ligeramente superior.

LightGBM:
Obtuvo el mejor accuracy (0.9690) con un árbol de profundidad 5 y 100 estimadores.



Ejecución del mejor modelo optimizado (LightGBM)

In [84]:
# Modelo con los mejores parámetros encontrados
best_lgbm = LGBMClassifier(
    max_depth=5,
    n_estimators=100,
    random_state=42
)

In [85]:
# Pipeline con preprocesador y modelo optimizado
pipeline_best_lgbm = Pipeline(steps=[
    ("preprocessor", preprocessor_lgbm),
    ("classifier", best_lgbm)
])

In [86]:
# Entrenar
pipeline_best_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 3873, number of negative: 10946
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 71
[LightGBM] [Info] Number of data points in the train set: 14819, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.261354 -> initscore=-1.038945
[LightGBM] [Info] Start training from score -1.038945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Time_spent_Alone',
                                                   'Social_event_attendance',
                                                   'Going_outside',
                                                   'Friends_circle_size',
                                                   'Post_frequency']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Stage_fear',
                                                   'Drained_after_socializing'])])),
                ('classifier', LGBMClassifier(max_depth=5, random_state=42))])

In [87]:
# Predecir
y_pred_lgbm = pipeline_best_lgbm.predict(X_test)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [88]:
# Evaluar
accuracy_lgbm = accuracy_score(y_test, y_pred_lgbm)

In [89]:
# Resultado
print(f"Accuracy LightGBM: {accuracy_lgbm:.4f}")

Accuracy LightGBM: 0.9690


**CONCLUSION FINAL**

LightGBM se posiciona como el modelo con mejor desempeño, aunque la diferencia con KNN y Regresión Logística es mínima.

La mayor mejora se dio en el Árbol de Decisión, fue clave la optimización, ya que subió mucho su accuracy con ajuste de profundidad.

Los modelos más fuertes (KNN, Regresión Logística, LightGBM) tenían ya una base muy buena, por eso la optimización solo trajo pequeños cambios o incluso ligeras variaciones.

El modelo de machine learning entrenado logró una alta precisión (alrededor del 97%) para distinguir entre personas introvertidas y extrovertidas a partir de características relacionadas con su comportamiento social (como tiempo pasado a solas, asistencia a eventos sociales, círculo de amigos, entre otros).

Esto indica que las variables analizadas capturan efectivamente las diferencias en hábitos y patrones sociales que caracterizan a cada tipo de personalidad.